In [11]:
import cv2
import mediapipe as mp
import numpy as np
import logging
import cProfile
cv2.setNumThreads(1)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

In [12]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    # define print method
    
    def __str__(self):
        return f'({self.x}, {self.y})'

def calculate_distance(point1, point2):
    return np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
                # uncomment if 3d Point
                #    + (point1['z'] - point2['z'])**2)
                
def calculate_horizontal_distance(point1, point2, frame_width):
    return abs(point1.x - point2.x) * frame_width

def calculate_vertical_distance(point1, point2, frame_height):
    return abs(point1.y - point2.y) * frame_height

def calculate_central_point(landmark1, landmark2):
    return Point((landmark1.x + landmark2.x) / 2, (landmark1.y + landmark2.y) / 2)

def low_pass_filter(new_value, prev_value, alpha=0.8):
    return alpha * new_value + (1 - alpha) * prev_value

def calculate_foot_angle(ankle, toe):
    """
    Calculate the angle of the foot relative to the ground.

    Parameters:
        ankle (Point): The position of the ankle (with x, y coordinates).
        toe (Point): The position of the toe (with x, y coordinates).

    Returns:
        float: The angle of the foot in degrees.
    """
    delta_x = abs(ankle.x - toe.x)
    delta_y = abs(ankle.y - toe.y)

    # Calculate the angle in radians
    angle_radians = np.arctan2(delta_y, delta_x)

    # Convert the angle to degrees
    angle_degrees = abs(angle_radians * 180.0 / np.pi)
    
    if angle_degrees > 180.0:
        angle_degrees = 360.0 - angle_degrees

    return angle_degrees

def calculate_foot_slope(ankle, toe):
    slope = (toe.y - ankle.y) / (toe.x - ankle.x)
    logging.info(f"Right foot slope: {slope}")

    return slope

def is_foot_on_ground(side, prev_toe, curr_toe, ground_line_y):
    toe_status_message = 'Toe approaching ground' if curr_toe.y < prev_toe.y else 'Toe leaving ground'
    low_pass_curr_toe_y = low_pass_filter(curr_toe.y, prev_toe.y)

    logging.info(f'{side} curr_toe: ({curr_toe.x}, {curr_toe.y}), {toe_status_message}')
    logging.info(f'{side} point_y: {curr_toe.y}, ground_line_y: {ground_line_y}, {side} leg on ground: {low_pass_curr_toe_y < ground_line_y}')
    return curr_toe.y < prev_toe.y and low_pass_curr_toe_y < ground_line_y

# def get_actual_height_in_px()

# print(calculate_central_point(Point(1, 1), Point(2, 2)))
# print(calculate_distance(Point(1, 1), Point(2, 2)))

In [13]:
def gait_speed_walk_overall(
    video_path,
    person_height_in_cm,
    distance_required_in_cm,
    movement_threshold=0.02,
    debug=True,
):
    logging.info("Starting video processing")

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        logging.error("Cannot determine the frame rate (FPS) of the video.")
        return 0, 0, 0  # Default values for stride length, elapsed time, and speed

    start_frame_id = None
    end_frame_id = None
    start_position = None
    distance_walked = 0
    timer_started = False
    initial_position = None
    strides = []
    
    central_point = None

    pixels_to_cm_ratio = None
    start_line = None
    ground_line = None

    ground_threshold = 0.08
    start_line_offset = 0.01

    previous_gray = None
    previous_keypoints = None
    foot_contact_start = False  # Flag to track first contact
    foot_off_ground = False  # Flag to track if foot is lifted off ground
    reset_needed = False  # Flag to reset after stride is recorded
    initial_foot_slope = None  # Initial slope of the foot


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            logging.warning("End of video or cannot read the video file")
            break

        current_frame_id = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(frame_rgb)

        current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        logging.info(
            f"===================== Processing frame {current_frame_id} =================================="
        )
        if results.pose_landmarks:
            # Draw pose landmarks
            if debug:
                mp_drawing.draw_landmarks(
                    frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                )

            landmarks = results.pose_landmarks.landmark
            # Convert NormalizedLandmark to Point
            left_hip = Point(
                landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP].y,
            )
            right_hip = Point(
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y,
            )
            nose = Point(
                landmarks[mp_pose.PoseLandmark.NOSE].x,
                landmarks[mp_pose.PoseLandmark.NOSE].y,
            )
            left_ankle = Point(
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].x,
                landmarks[mp_pose.PoseLandmark.LEFT_ANKLE].y,
            )
            right_ankle = Point(
                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE].y,
            )
            left_toe = Point(
                landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].x,
                landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX].y,
            )
            right_toe = Point(
                landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].x,
                landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX].y,
            )
            keypoints = {
                "LEFT_HIP": left_hip,
                "RIGHT_HIP": right_hip,
                "NOSE": nose,
                "LEFT_ANKLE": left_ankle,
                "RIGHT_ANKLE": right_ankle,
                "LEFT_TOE": left_toe,
                "RIGHT_TOE": right_toe,
            }
            keypoint_array = np.array(
                [[kp.x * frame.shape[1], kp.y * frame.shape[0]] for kp in keypoints.values()],
                dtype=np.float32
            ).reshape(-1, 1, 2)

            central_ankle = calculate_central_point(left_ankle, right_ankle)
            logging.info(f"Central ankle point: {central_ankle}")

            # Calculate the height of the person in px
            estimated_height_in_px = calculate_vertical_distance(nose, central_ankle, frame.shape[0])   # Possibly add a offset to the height

            if pixels_to_cm_ratio is None:
                pixels_to_cm_ratio = estimated_height_in_px / person_height_in_cm
                logging.info(f"Pixels to cm ratio: {pixels_to_cm_ratio} is set")

            # Fix central point of the person using hip points
            central_point = calculate_central_point(left_hip, right_hip)
            logging.info(f"Central point: {central_point}")

            if ground_line is None:
                # Set the ground line slightly below the ankle
                ground_line = central_ankle.y + ground_threshold
                logging.info(f"Ground line set at Y: {ground_line}")
            else:
                if debug:
                    cv2.line(
                        frame,
                        (0, int(ground_line * frame.shape[0])),
                        (frame.shape[1], int(ground_line * frame.shape[0])),
                        (0, 255, 0),  # Green color for the ground line
                        2,
                    )

            if initial_position is None and central_point is not None:
                initial_position = central_point
                logging.info(f"Initial position set: {initial_position}")

                # Set start line slightly in front of the initial frontmost toe
                start_line = max(left_toe.x, right_toe.x) + start_line_offset
                logging.info(f"Start line set at X: {start_line}")

            # Draw the start line
            if debug and start_line is not None:
                # Convert normalized start line X position to pixel coordinates
                start_line_pixel_x = int(start_line) * frame.shape[1]

                cv2.line(
                    frame,
                    (start_line_pixel_x, 0),
                    (start_line_pixel_x, frame.shape[0]),
                    (255, 0, 0),  # Blue color for the start line
                    2,
                )

            if not timer_started:
                # Check if either toe crosses the start line
                if max(left_toe.x, right_toe.x) >= start_line:
                    start_position = central_point
                    start_frame_id = current_frame_id
                    timer_started = True
                    logging.info(f"Timer started at frame {start_frame_id}")

            # Main Processing only when the timer has started
            if timer_started:
                distance_walked = (
                    calculate_horizontal_distance(start_position, central_point, frame.shape[1])
                    / pixels_to_cm_ratio
                )
                logging.info(f"Distance walked: {distance_walked} cm")

                # End processing
                if distance_walked >= distance_required_in_cm:
                    end_frame_id = current_frame_id
                    logging.info(
                        f"Distance: {distance_required_in_cm} cm is completed - {distance_walked} cm"
                    )
                    break
                
                
                
            # Display timer and distance
            if timer_started and start_frame_id is not None:
                elapsed_time = (current_frame_id - start_frame_id) / fps
                if end_frame_id is not None:
                    distance_walked = (
                        calculate_horizontal_distance(start_position, central_point, frame.shape[1])
                        / pixels_to_cm_ratio
                    )
                    elapsed_time = (end_frame_id - start_frame_id) / fps
                if debug:
                    cv2.putText(
                        frame,
                        f"Time: {elapsed_time:.2f} sec",
                        (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA,
                    )
                    cv2.putText(
                        frame,
                        f"Distance: {distance_walked:.2f} cm",
                        (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 255, 0),
                        2,
                        cv2.LINE_AA,
                    )
                    
            else:
                if debug:
                    cv2.putText(
                        frame,
                        f"Timer not started",
                        (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1,
                        (0, 0, 255),
                        2,
                        cv2.LINE_AA,
                    )
            
            if not timer_started and initial_position is None:
                logging.info("Waiting for the person to start walking")

            if previous_gray is not None and previous_keypoints is not None:
                p1, st, err = cv2.calcOpticalFlowPyrLK(
                    previous_gray, current_gray, previous_keypoints, None
                )

                for i, (new, old) in enumerate(zip(p1, previous_keypoints)):
                    new_x, new_y = new.ravel()
                    old_x, old_y = old.ravel()

                    motion_magnitude = np.sqrt((new_x - old_x) ** 2 + (new_y - old_y) ** 2)
                    motion_angle = np.arctan2(new_y - old_y, new_x - old_x)

                    key = list(keypoints.keys())[i]
                    logging.info(
                        f"{key} - Magnitude: {motion_magnitude}, Angle: {motion_angle} radians"
                    )

                    # Optionally, visualize the motion (debugging)
                    if debug:
                        cv2.arrowedLine(
                            frame,
                            (int(old_x), int(old_y)),
                            (int(new_x), int(new_y)),
                            (0, 255, 0),
                            2,
                            tipLength=0.5
                        )

                previous_keypoints = p1
            else:
                previous_keypoints = keypoint_array
            
            previous_gray = current_gray
            
            if initial_foot_slope is None:
                initial_foot_slope = calculate_foot_slope(keypoints["RIGHT_ANKLE"], keypoints["RIGHT_TOE"])
                right_slope = calculate_foot_slope(keypoints["RIGHT_ANKLE"], keypoints["RIGHT_TOE"])
                logging.info(f"Initial foot slope: {initial_foot_slope}")
            else:
                right_slope = calculate_foot_slope(keypoints["RIGHT_ANKLE"], keypoints["RIGHT_TOE"])
                logging.info(f"Right foot slope: {right_slope}")
            
            if abs(right_slope) < initial_foot_slope:
                logging.info(f"Right foot is parallel to the ground")
                if not foot_contact_start and not reset_needed:
                    # First ground contact
                    foot_contact_start = True
                    foot_off_ground = False
                    initial_contact_position = right_toe
                    logging.info("First ground contact detected")

                elif foot_off_ground and reset_needed:
                    # Second ground contact, calculate stride
                    final_contact_position = right_toe
                    stride_length = (
                        calculate_horizontal_distance(
                            final_contact_position, initial_contact_position, frame.shape[1]
                        )
                        / pixels_to_cm_ratio
                    )
                    strides.append(stride_length)
                    logging.info(f"Stride length recorded: {stride_length} cm")

                    # Reset flags for the next stride detection
                    reset_needed = False
                    foot_contact_start = False
                    foot_off_ground = False
                    initial_contact_position = right_toe
            else:
                # Foot off ground
                if foot_contact_start and not reset_needed:
                    foot_off_ground = True
                    logging.info("Foot is off the ground")

                    # Allow reset to detect the next stride
                    reset_needed = True

            logging.info(f"Right foot slope: {right_slope}")
            cv2.putText(
                frame,
                f"Right foot slope: {right_slope}",
                (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),
                2,
                cv2.LINE_AA,
            )
            if abs(right_slope) < 0.9:
                logging.info(f"Right foot is parallel to the ground")

            cv2.putText(
                frame,
                f"Foot touching ground: {'YES' if foot_contact_start and abs(right_slope) < 0.9 else 'NO'}",
                (10, 120),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 0),  # Yellow text for stride count
                2,
                cv2.LINE_AA,
            )
        if debug:
            # Display the frame
            cv2.imshow("Frame", frame)
            if cv2.waitKey(5) & 0xFF == 27:
                logging.info("Process interrupted by user")
                break
    
    cap.release()
    cv2.destroyAllWindows()
    logging.info("================================= Video processing completed =================================")
    if strides:
        average_stride_length = np.mean(strides)
        logging.info(f"strides: {strides}")
        logging.info(f"Average stride length: {average_stride_length} cm")
        strides_per_second = len(strides) / elapsed_time if elapsed_time > 0 else 0
        logging.info(f"Strides per second: {strides_per_second}")
    else:
        average_stride_length = 0
        logging.info("No strides detected")

    if timer_started and start_frame_id is not None:
        elapsed_time = (current_frame_id - start_frame_id) / fps
        logging.info(f"Elapsed time: {elapsed_time} seconds")
        average_speed = distance_walked / elapsed_time if elapsed_time > 0 else 0
        logging.info(f"Average speed: {average_speed} cm/second")
    else:
        elapsed_time = 0
        average_speed = 0
        logging.info("Problems detected, unable to calculate")

    return distance_walked,elapsed_time, average_speed, strides, average_stride_length


# video_path = "/Users/brennanlee/Desktop/opencv-healthcare/test/GSWT(forward).mp4"
# video_path = "/Users/brennanlee/Desktop/opencv-healthcare/test/GSWT_normal_youlian.mp4"
# video_path = "/Users/brennanlee/Desktop/opencv-healthcare/test/GSWT_fast_youlian.mp4"
video_path = "/Users/brennanlee/Desktop/opencv-healthcare/test/GSWT_frail_youlian.mp4"
person_height = 175  # Example height in meters
strides, elapsed_time, average_speed, distance_walked, average_stride_length = gait_speed_walk_overall(
    video_path, person_height, 400
)

cProfile.run('gait_speed_walk_overall(video_path, person_height, 400)')



2024-09-08 01:29:48,422 - INFO - Starting video processing
2024-09-08 01:29:48,744 - INFO - ===================== Processing frame 1 ==================================
2024-09-08 01:29:48,949 - INFO - ===================== Processing frame 2 ==================================
2024-09-08 01:29:48,951 - INFO - Central ankle point: (0.07096335291862488, 0.9301642775535583)
2024-09-08 01:29:48,951 - INFO - Pixels to cm ratio: 6.948514979226249 is set
2024-09-08 01:29:48,952 - INFO - Central point: (0.09554227069020271, 0.6560676097869873)
2024-09-08 01:29:48,952 - INFO - Ground line set at Y: 1.0101642775535584
2024-09-08 01:29:48,953 - INFO - Initial position set: (0.09554227069020271, 0.6560676097869873)
2024-09-08 01:29:48,953 - INFO - Start line set at X: 0.12150011420249938
2024-09-08 01:29:48,954 - INFO - Right foot slope: 0.7672596224639006
2024-09-08 01:29:48,954 - INFO - Right foot slope: 0.7672596224639006
2024-09-08 01:29:48,955 - INFO - Initial foot slope: 0.7672596224639006
20

In [7]:
print(np.sum(strides))
print(np.mean(strides))
print(strides)
# Time to walk 263.9982179283159 cm: 3.100798995150204 seconds


207.16048206369624
51.79012051592406
[93.76797217101132, 97.59619489779173, 2.0364751939196752, 13.759839800973515]
